### **Content License Agreement**

<font color='red'><b>**WARNING**</b></font> : 본 자료는 삼성청년SW·AI아카데미의 컨텐츠 자산으로, 보안서약서에 의거하여 어떠한 사유로도 임의로 복사, 촬영, 녹음, 복제, 보관, 전송하거나 허가 받지 않은 저장매체를 이용한 보관, 제3자에게 누설, 공개 또는 사용하는 등의 무단 사용 및 불법 배포 시 법적 조치를 받을 수 있습니다.

# **Objectives**



**1. 실습 개요**

* **목표:** 학습된 `Pytorch` 모델을 온디바이스 배포용 `TFLite` 모델로 변환 및 최적화.
* **핵심 과제:**
    1.  **모델 재구성:** 표준 Hugging Face 모델을 `TFLite` 변환이 가능한 `ai-edge-torch` 라이브러리 기반 아키텍처로 재설계.
    2.  **가중치 이식:** 원본 모델과 재구성된 모델 간 아키텍처 불일치를 해결하고, 가중치를 수동으로 매핑하는 커스텀 함수 구현.
    3.  **성능 저하 해결:** 모델 재구성 과정에서 발생하는 미세 설정 차이를 분석하여 예측 성능 저하 문제 해결.



**2. 실습 목적 및 배경**

* **온디바이스 최적화 이해:** 표준 모델을 온디바이스용으로 재구성해야 하는 근본적인 이유 학습.
* **아키텍처 분석 능력 함양:** 표준 트랜스포머와 온디바이스 최적화 모델(예: Fused QKV Layer)의 구조적 차이점 분석.
* **온디바이스 AI 파이프라인 경험:** 모델 재구성, 가중치 이식, `TFLite` 변환, 최종 애플리케이션 배포까지 전 과정 이해.
* **양자화(Quantization) 효과 이해:** 모델 경량화 및 추론 속도 향상, NPU 환경에서 `INT8` 연산의 성능 향상 원리 파악.


**3. 획득 가능 역량**

* **On-device AI 모델 최적화 역량:** 특정 하드웨어 환경에 맞게 모델을 재구성하고 양자화하여 성능을 극대화하는 실무 능력.
* **엔드투엔드(End-to-End) 프로젝트 수행 역량:** 모델 선정부터 최적화, 변환, 배포까지 **On-device AI** 개발 파이프라인 전체를 완수하는 능력.



**4. 실습 핵심 내용**

* `SmolLM2` 모델 재구성 및 `TFLite` (LiteRT) 변환.
* `TFLite Interpreter`를 이용한 추론 실행.

# **Prerequistes**


**[Prerequistes]**

```bash
ai_edge_torch: 0.4.0
tensorflow: 2.19.0
torch: 2.6.0+cu124
torchvision: 0.21.0
torchaudio: 2.6.0
```


**[install 명령어]**

- colab에 ai_edge_torch 는 설치가 되어있지 않을 확률이 높기 때문에 해당 버전을 유의해서 설치해주세요.
- tensorflow 및 torch 와의 버전이 맞아 떨어져야하기 때문에 두 개의 설치 명령어를 같이 실행해주세요.
- 설치 도중 코랩에서 기본적으로 설치된 라이브러리와 새로 설치하는 라이브리리 사이에서 발생하는 의존성 문제가 발생할 수 있습니다. 이는 Python 패키지 관리 구조상, 특정 버전 조합이 완벽히 호환되지 않는 경우가 많기에 발생하고, 강의 실습에 큰 영향을 주지 않는 단순 Error이니 안심하고 실습을 진행해주셔도 됩니다.


```bash
! pip install tensorflow==2.19.0
! pip install ai_edge_torch==0.4.0
! pip install torch==2.6.0 torchvision==0.21.0 torchaudio==2.6.0 --index-url https://download.pytorch.org/whl/cu124

```

In [21]:
import torch
print(torch.__version__)

import tensorflow
print(tensorflow.__version__)

import ai_edge_torch
print(ai_edge_torch.__version__)

# 버전이 다르거나 설치되어있지 않다면 아래 명령어를 실행하도록 합니다.
# ! pip install tensorflow==2.19.0
# ! pip install ai_edge_torch==0.4.0
# ! pip install torch==2.6.0 torchvision==0.21.0 torchaudio==2.6.0 --index-url https://download.pytorch.org/whl/cu124

2.6.0+cu124
2.19.0
0.4.0


In [22]:
# ! pip install tensorflow==2.19.0
# ! pip install ai_edge_torch==0.4.0
# ! pip install torch==2.6.0 torchvision==0.21.0 torchaudio==2.6.0 --index-url https://download.pytorch.org/whl/cu124


# **Exercise Overview**

### **들어가며: LiteRT (이전 TFLite)**
LiteRT는 구글이 제공하는 온디바이스 AI 실행 엔진으로 원래는 TensorFlow Lite(TFLite)라는 이름으로 널리 쓰였는데, 2024년 이름이 Lite Runtime(LiteRT)으로 바뀌어졌습니다.

단순히 이름만 바뀐 게 아니라, 앞으로는 TensorFlow 프레임워크에 종속되지 않고 PyTorch, JAX, Keras 등 다양한 프레임워크에서 만든 모델을 실행할 수 있는 범용 런타임으로 확장하려는 의도가 담겨 있습니다.

주요 특징으로는 다음이 있습니다.

- 온디바이스 AI 실행
  - 모바일(Android, iOS), IoT, 임베디드 기기에서 AI 모델을 가볍게 실행할 수 있음.
  - 클라우드 서버 없이 동작 → 지연시간 감소, 프라이버시 강화, 네트워크 의존도 줄어듦.

- 다양한 프레임워크 지원
  - 과거 TFLite는 TensorFlow에서 변환한 모델만 잘 지원했음.
  - LiteRT는 PyTorch, JAX, Hugging Face 모델 등도 변환해 실행 가능.

- 최적화 기능

- 모델 경량화(Quantization: 정수/반정밀도 변환)

  - 연산 최적화 및 하드웨어 가속기(TPU, GPU, NPU, DSP 등) 지원
  - 광범위한 디바이스 지원
    - Android / iOS
    - Raspberry Pi, 마이크로컨트롤러(MCU) 같은 초저전력 디바이스까지 지원

### **들어가며: AI Edge Torch**

PyTorch 모델을 LiteRT(Lite Runtime, 구 TensorFlow Lite) 포맷으로 변환·최적화하기 위한 Python SDK 입니다.
즉, PyTorch 개발자가 별도의 TensorFlow 변환 과정을 거치지 않고도 PyTorch 모델을 바로 LiteRT로 내보낼 수 있게 해주는 도구입니다.

기존에는 PyTorch → ONNX → TFLite 변환 같은 복잡한 경로가 필요했는데, 이를 단순화 했습니다.


### **실습 목차**

이 노트북은 두 가지 주요 파트로 구성되어 있습니다.
1. Sequence Classification 모델 변환
2. 변환된 모델을 tflite interpreter 을 통해 실행


# **실습1: Sequence Classification 모델 재구성 및 변환**

해당 파트에서는 ai-edge-torch 라이브러리를 사용하여 사전 학습된 트랜스포머(Transformer) 모델을 시퀀스 분류(Sequence Classification) 작업에 맞게 수정한 뒤, TensorFlow Lite (TFLite) 형식으로 변환하는 전체 과정을 단계별로 알아봅니다.

TFLite로 변환하면 모델을 모바일이나 엣지 디바이스에 배포하여 더 빠르고 효율적으로 추론을 수행할 수 있습니다.


먼저, 모델을 정의하고 변환하는 데 필요한 **라이브러리들을 임포트**합니다.

In [23]:
# --- PyTorch 및 기본 라이브러리 ---
import torch
from torch import nn
from typing import Optional, Dict
import numpy as np
import os
import torch
from collections import OrderedDict
import safetensors.torch

# --- transformer 라이브러리 ---
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    AutoModelForSequenceClassification
)

# --- ai_edge_torch 관련 라이브러리 ---
# 모델 변환 및 양자화를 위한 핵심 도구
import ai_edge_torch

# 생성 모델 레이어 (Attention, TransformerBlock 등) 및 설정을 위한 유틸리티
from ai_edge_torch.generative.layers import attention, builder
import ai_edge_torch.generative.layers.attention_utils as attn_utils
import ai_edge_torch.generative.layers.model_config as cfg

# 사전 학습된 가중치를 불러오기 위한 로더 유틸리티
import ai_edge_torch.generative.utilities.loader as loading_utils
from safetensors import safe_open

# quantization 을 위한 유틸리티
from ai_edge_torch.generative.quantize import quant_recipes

# --- google drive 연결 ---
from google.colab import drive
drive.mount('/content/drive')
root='/content/drive/MyDrive'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


다음으로 **시퀀스 분류 모델을 정의**합니다.

### `ai_edge_torch`를 사용해 모델을 새로 정의하는 이유

* **온디바이스(on-device) 환경**에 최적화된, 가볍고 효율적인 `TFLite` 모델을 생성하기 위함.
* **HuggingFace:** 연구/개발 편의성 및 범용성 초점.
* **`ai_edge_torch`:** 모바일 기기 등 제한된 환경에서의 **추론 성능 극대화** 초점.


### 핵심 차이점

#### 1. 타겟 환경의 차이: 서버 vs 엣지 디바이스

* **HuggingFace `transformers`**
    * **타겟:** 서버, PC 환경 (GPU).
    * **우선순위:** 개발 편의성, 모델 유연성, 쉬운 학습.
    * **특징:** `forward` 함수 내 다양한 분기문 및 동적 로직 포함 가능.
* **`ai_edge_torch`**
    * **타겟:** 모바일, IoT 등 엣지 디바이스.
    * **고려사항:** 제한된 메모리, 연산 능력, 배터리.
    * **요구사항:** 모든 구성 요소(레이어, 연산)가 `TFLite` 변환 및 모바일 하드웨어(NPU, DSP) 가속에 최적화되어야 함.

#### 2. 모델 구조의 최적화 수준

* **Hugging Face `transformers`**
    * **구조:** `PyTorch` 표준 레이어(`nn.Linear`, `nn.LayerNorm` 등)와 파이썬 로직을 자유롭게 사용.
    * **한계:** 일부 연산은 `TFLite` 변환 시 모바일 하드웨어에서 비효율적이거나 미지원.
* **`ai_edge_torch`**
    * **구조:** `TFLite` 변환 및 모바일 가속기에 최적화된 레이어만 사용하여 모델 "재조립".
    * **특징:** 범용성보다 성능을 위해 의도적으로 모델 구조와 사용 가능 연산을 제한.

#### 3. 정적 그래프(Static Graph)와 추적(Tracing)

* **`TFLite` 변환 전제조건:** 모델의 연산 흐름이 **'정적 그래프(Static Graph)'** 형태로 고정되어야 함.
* **Hugging Face `transformers`**
    * `forward` 함수가 복잡하고 분기 처리가 많아 정적 그래프로 변환(추적)이 매우 어렵거나 불가능.
* **`ai_edge_torch`**
    * 모델 정의 단계부터 모든 레이어와 로직을 `TFLite` 변환기가 쉽게 추적하도록 간결하게 설계.
    * 이것이 `ai_edge_torch` 부품으로 모델을 새로 만드는 ***가장 큰 이유*** 중 하나.

이전에 불러왔던 모델의 형태는 다음과 같습니다.
SmolLM 모델로 `AutoModelForSequenceClassification` 을 생성하면 내부적으로 `LlamaForSequenceClassification` 을 호출하기 때문에 현재 이름이 `Llama` 으로 되어있습니다.

```
LlamaForSequenceClassification(
  (model): LlamaModel(
    (embed_tokens): Embedding(49152, 576, padding_idx=2)
    (layers): ModuleList(
      (0-29): 30 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=576, out_features=576, bias=False)
          (k_proj): Linear(in_features=576, out_features=192, bias=False)
          (v_proj): Linear(in_features=576, out_features=192, bias=False)
          (o_proj): Linear(in_features=576, out_features=576, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=576, out_features=1536, bias=False)
          (up_proj): Linear(in_features=576, out_features=1536, bias=False)
          (down_proj): Linear(in_features=1536, out_features=576, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((576,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((576,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((576,), eps=1e-05)
    (rotary_emb): LlamaRotaryEmbedding()
  )
  (score): Linear(in_features=576, out_features=2, bias=False)
)
```

다음으로 이에 맞춰 sequence classification model 을 정의해봅시다.

이 함수에서는 SmolLM2이라는 특정 트랜스포머 모델의 아키텍처를 정의하고 sequence classification model 에 맞게 추가로 score layer 을 추가합니다. (앞 모듈에서 사용하였던 모델입니다).

In [24]:
class SequenceClassificationModel(nn.Module):
    """
    사전 학습된 트랜스포머 아키텍처를 기반으로 하는 텍스트 분류 모델입니다.
    TFLite와 같은 엣지 디바이스 환경으로의 변환을 염두에 두고 설계되었습니다.
    모델은 입력된 텍스트 시퀀스 전체의 의미를 압축하여 특정 클래스로 분류하는 작업을 수행합니다.
    """
    def __init__(self, config: cfg.ModelConfig, num_classes: int = 2):
        """
        모델의 레이어를 초기화합니다.

        Args:
            config (cfg.ModelConfig): 모델의 구조와 하이퍼파라미터를 담고 있는 설정 객체.
            num_classes (int): 분류할 클래스의 개수.
        """
        super().__init__()
        self.config = config

        # --- 모델의 주요 구성 요소 ---

        # --- 1. 토큰 임베딩 레이어 ---
        # 정수로 된 토큰 ID를 고차원의 벡터 표현으로 변환합니다.
        # 이 벡터는 단어의 의미를 내포하게 됩니다.
        self.tok_embedding = nn.Embedding(
            config.vocab_size, config.embedding_dim, padding_idx=2,
        )

        # --- 2. 트랜스포머 블록 ---
        # 모델의 핵심 엔진으로, 여러 개의 트랜스포머 블록이 쌓여 있습니다.
        # 각 블록은 셀프 어텐션(self-attention) 메커니즘을 통해
        # 문장 내 단어들의 문맥적 관계를 학습합니다.
        self.transformer_blocks = nn.ModuleList(
            attention.TransformerBlock(config.block_config(idx), config)
            for idx in range(config.num_layers)
        )

        # --- 3. 최종 정규화 레이어 ---
        # 모든 트랜스포머 블록을 통과한 출력은 불안정할 수 있으므로,
        # 마지막으로 정규화를 적용하여 안정적인 값을 얻습니다.
        self.final_norm = builder.build_norm(
            config.embedding_dim, config.final_norm_config
        )

        # --- 4. 어텐션 마스크 캐시 ---
        # 디코더-온리 모델의 특성상, 각 토큰이 자기 자신보다 뒤에 있는 토큰을
        # 볼 수 없도록 가려주는 '인과 관계 마스크(causal mask)'가 필요합니다.
        # 이 마스크를 미리 계산하여 캐싱해두면 추론 속도를 높일 수 있습니다.
        self.mask_cache = attn_utils.build_causal_mask_cache(
            size=config.kv_cache_max,
        )

        # --- 5. 분류 헤드 (Classification Head) ---
        # 트랜스포머가 추출한 문장 전체의 의미 벡터를 입력받아,
        # 최종적으로 각 클래스에 대한 점수(logit)를 출력하는 선형 레이어입니다.

        # 문제 1: Classification 을 위해 score 레이어를 정의합니다.
        # 이전 모듈에서 생성한 모델과 동일해야하므로 이름을 score 으로 지정합니다. bias 는 없습니다.
        # [START CODE]
        self.score = nn.Linear(
            config.embedding_dim, num_classes, bias=False
        )
        # [END CODE]

    @torch.inference_mode()
    def forward(self, input_ids: torch.Tensor, attention_mask: torch.Tensor) -> torch.Tensor:
        # --- 0. 초기 설정 ---
        # 입력 텐서로부터 배치 크기와 시퀀스 길이를 가져옵니다.
        batch_size, seq_len = input_ids.size()

        # --- 1. 임베딩 생성 ---
        # 입력된 토큰 ID를 고차원의 벡터 표현(임베딩)으로 변환합니다.
        input_embeds = self.tok_embedding(input_ids)

        # --- 2. RoPE (회전 위치 임베딩) 준비 ---
        # 모델 설정에서 어텐션 관련 설정을 가져옵니다.
        attn_config = self.config.block_config(0).attn_config
        # RoPE를 적용할 임베딩 차원의 크기를 계산합니다.
        n_elem = int(attn_config.rotary_percentage * attn_config.head_dim)
        # 시퀀스 길이에 맞는 위치 인덱스([0, 1, 2, ...])를 생성합니다.
        input_pos = torch.arange(0, seq_len, device=input_ids.device)
        # 위치 인덱스를 기반으로 RoPE 텐서를 생성합니다. 이는 모델이 토큰의 순서 정보를 학습하는 데 사용됩니다.
        rope = self.config.build_rope(input_pos, n_elem, attn_config.rotary_base)

        # --- 3. 어텐션 마스크 결합 ---
        # 미리 계산된 인과 관계 마스크(causal mask) 캐시에서 현재 시퀀스 길이에 맞는 부분을 가져옵니다.
        # 인과 관계 마스크는 각 토큰이 자기 자신과 이전 토큰에만 집중하도록 제한합니다.
        causal_mask = self.mask_cache.index_select(2, input_pos)
        causal_mask = causal_mask[:, :, :seq_len, :seq_len]

        # attention_mask에서 패딩(padding) 부분(값이 0)을 찾아 boolean 마스크를 생성합니다.
        padding_mask_bool = (attention_mask == 0)
        # 패딩 마스크의 차원을 (batch, 1, 1, seq_len) 형태로 확장하여
        # 어텐션 마스크 (batch, num_heads, seq_len, seq_len)와 연산이 가능하도록 만듭니다.
        padding_mask_bool = padding_mask_bool.unsqueeze(1).unsqueeze(2)
        # 인과 관계 마스크에 패딩 마스크를 결합합니다. 패딩 위치는 -무한대로 채워져 어텐션 계산에서 제외됩니다.
        mask = causal_mask.masked_fill(padding_mask_bool, -torch.inf)

        # --- 4. 트랜스포머 블록 통과 ---
        x = input_embeds
        # 설정에 따라 임베딩 벡터에 스케일링을 적용할 수 있습니다.
        if self.config.embedding_scale is not None:
            x = x * self.config.embedding_scale

        # 준비된 입력(x), RoPE, 최종 마스크를 모든 트랜스포머 블록에 순차적으로 통과시킵니다.
        for block in self.transformer_blocks:
            # kv_cache 인자(None, None)는 텍스트 생성 시에만 사용되므로 여기서는 비워둡니다.
            x = block(x, rope, mask, input_pos, None, None)

        # 모든 블록을 통과한 최종 결과에 정규화(Normalization)를 적용합니다.
        x = self.final_norm(x)

        # --- 5. 풀링(Pooling) 및 분류 ---
        # attention_mask의 합계를 통해 각 시퀀스의 실제 길이를 계산합니다.
        actual_seq_lengths = attention_mask.sum(dim=1)
        # 실제 마지막 토큰의 인덱스를 계산합니다. (길이 - 1)
        last_token_indices = actual_seq_lengths - 1
        # 배치 내 각 항목을 선택하기 위한 인덱스를 생성합니다. (예: [0, 1, 2, ...])
        batch_indices = torch.arange(batch_size, device=input_ids.device)
        # x에서 각 시퀀스의 '실제 마지막 토큰'에 해당하는 은닉 상태 벡터만 추출합니다.
        # 이 벡터가 문장 전체의 의미를 대표하는 벡터로 사용됩니다.
        pooled_output = x[batch_indices, last_token_indices]

        # 문제 2: 추출된 대표 벡터를 분류 레이어(score)에 통과시켜 최종 로짓(logits)을 계산합니다.
        # [START CODE]
        logits = self.score(pooled_output)
        # [END CODE]

        # --- 6. 결과 반환 ---
        # 최종 로짓을 딕셔너리 형태로 반환합니다.
        return {'logits': logits}

다음은 모델의 상세한 구조(레이어 수, 임베딩 차원 등)를 정의하는 `get_model_config` 함수를 작성합니다.
 어텐션 헤드 개수, 레이어 수 등 모델의 모든 하이퍼파라미터가 여기에 명시됩니다. 해당 하이퍼파라미터들은 config.json 에서 확인할 수 있습니다.


In [25]:
def get_model_config() -> cfg.ModelConfig:
    """SmolLM 135M 모델의 설정을 반환합니다."""

    # 문제 3: 모델 생성을 위한 config 을 작성합니다.
    # 모델이 저장된 경로의 config.json 을 참고하여 작성하세요.
    # [START CODE]

    attn_config = cfg.AttentionConfig(
        num_heads=9,           # 어텐션 헤드의 수 (from config.json: num_attention_heads)
        head_dim=64,            # 각 헤드의 차원 (from config.json: head_dim)
        num_query_groups=3,    # Grouped Query Attention을 위한 그룹 수 (from config.json: num_key_value_heads)
        rotary_base=10000,         # from config.json: rope_theta
        rotary_percentage=1.0,
    )
    ff_config = cfg.FeedForwardConfig(
        type=cfg.FeedForwardType.GATED,
        activation=cfg.ActivationConfig(cfg.ActivationType.SILU),
        intermediate_size=1536, # 피드 포워드 네트워크의 중간 차원 (from config.json: intermediate_size)
    )
    norm_config = cfg.NormalizationConfig(type=cfg.NormalizationType.RMS_NORM,
                                          epsilon=1e-5)
    block_config = cfg.TransformerBlockConfig(
        attn_config=attn_config,
        ff_config=ff_config,
        pre_attention_norm_config=norm_config,
        post_attention_norm_config=norm_config,
    )
    config = cfg.ModelConfig(
        vocab_size=49152,        # 어휘 사전의 크기 (from config.json: vocab_size)
        num_layers=30,        # 트랜스포머 블록의 수 (from config.json: num_hidden_layers)
        max_seq_len=2048,       # 최대 시퀀스 길이
        embedding_dim=576,     # 임베딩 벡터의 차원 (from config.json: hidden_size)
        block_configs=block_config,
        final_norm_config=norm_config,
    )
    # [END CODE]

    config.block_config(0).attn_config.rotary_base = 100000
    return config

모델을 만들고 만든 모델을 반환해주는 `build_model` 함수를 작성합니다.

In [26]:
# build_sequence_classification_model을 더 간단히 호출하기 위한 래퍼(wrapper) 함수
def build_model(
        config: cfg.ModelConfig,
        model_class: type[nn.Module] = SequenceClassificationModel,
        num_classes: int = 2,
    ) -> nn.Module:
    transformer = model_class(config, num_classes=num_classes)
    transformer.eval()
    return transformer

In [27]:
# 문제 4: 작성한 함수를 호출하여 모델을 생성합니다. 생성한 모델을 출력하여 구조를 확인합니다.
# [START CODE]
# config 을 만듭니다.
config = get_model_config()
# model 을 만듭니다.
model = build_model(config)
# model 을 출력합니다.
model
# [END CODE]

SequenceClassificationModel(
  (tok_embedding): Embedding(49152, 576, padding_idx=2)
  (transformer_blocks): ModuleList(
    (0-29): 30 x TransformerBlock(
      (pre_atten_norm): RMSNorm()
      (atten_func): CausalSelfAttention(
        (qkv_projection): Linear(in_features=576, out_features=960, bias=False)
        (output_projection): Linear(in_features=576, out_features=576, bias=False)
      )
      (post_atten_norm): RMSNorm()
      (ff): GatedFeedForward(
        (w1): Linear(in_features=576, out_features=1536, bias=False)
        (w2): Linear(in_features=1536, out_features=576, bias=False)
        (w3): Linear(in_features=576, out_features=1536, bias=False)
      )
    )
  )
  (final_norm): RMSNorm()
  (score): Linear(in_features=576, out_features=2, bias=False)
)

다음으로 만들어낸 Pytorch 모델의 가중치를 이전에 미세조정한 모델의 가중치로 덮어씌웁니다.

먼저 이전에 미세조정한 모델의 가중치를 확인해봅니다.

In [28]:
# 자신의 경로에 맞게 경로를 수정합니다.
# 저장된 모델을 safetensor 형태로 불러와 weight 의 key 을 출력합니다.
smollm_checkpoint_path = f'{root}/smollm2_merged_for_inference'
tensors = safe_open(f'{smollm_checkpoint_path}/model.safetensors', framework="pt")

tensors.keys()

['model.embed_tokens.weight',
 'model.layers.0.input_layernorm.weight',
 'model.layers.0.mlp.down_proj.weight',
 'model.layers.0.mlp.gate_proj.weight',
 'model.layers.0.mlp.up_proj.weight',
 'model.layers.0.post_attention_layernorm.weight',
 'model.layers.0.self_attn.k_proj.weight',
 'model.layers.0.self_attn.o_proj.weight',
 'model.layers.0.self_attn.q_proj.weight',
 'model.layers.0.self_attn.v_proj.weight',
 'model.layers.1.input_layernorm.weight',
 'model.layers.1.mlp.down_proj.weight',
 'model.layers.1.mlp.gate_proj.weight',
 'model.layers.1.mlp.up_proj.weight',
 'model.layers.1.post_attention_layernorm.weight',
 'model.layers.1.self_attn.k_proj.weight',
 'model.layers.1.self_attn.o_proj.weight',
 'model.layers.1.self_attn.q_proj.weight',
 'model.layers.1.self_attn.v_proj.weight',
 'model.layers.10.input_layernorm.weight',
 'model.layers.10.mlp.down_proj.weight',
 'model.layers.10.mlp.gate_proj.weight',
 'model.layers.10.mlp.up_proj.weight',
 'model.layers.10.post_attention_layerno

다음으로 생성한 모델의 가중치를 확인해봅니다.

In [29]:
model.state_dict().keys()

odict_keys(['tok_embedding.weight', 'transformer_blocks.0.pre_atten_norm.weight', 'transformer_blocks.0.atten_func.qkv_projection.weight', 'transformer_blocks.0.atten_func.output_projection.weight', 'transformer_blocks.0.post_atten_norm.weight', 'transformer_blocks.0.ff.w1.weight', 'transformer_blocks.0.ff.w2.weight', 'transformer_blocks.0.ff.w3.weight', 'transformer_blocks.1.pre_atten_norm.weight', 'transformer_blocks.1.atten_func.qkv_projection.weight', 'transformer_blocks.1.atten_func.output_projection.weight', 'transformer_blocks.1.post_atten_norm.weight', 'transformer_blocks.1.ff.w1.weight', 'transformer_blocks.1.ff.w2.weight', 'transformer_blocks.1.ff.w3.weight', 'transformer_blocks.2.pre_atten_norm.weight', 'transformer_blocks.2.atten_func.qkv_projection.weight', 'transformer_blocks.2.atten_func.output_projection.weight', 'transformer_blocks.2.post_atten_norm.weight', 'transformer_blocks.2.ff.w1.weight', 'transformer_blocks.2.ff.w2.weight', 'transformer_blocks.2.ff.w3.weight', '

두 개의 가중치, 정확히는 가중치의 key 을 비교했을 때, key 가 다른것을 확인할 수 있습니다.
이렇게 key 가 다르면 가중치를 불러올 수 없기 때문에 이를 고려해서 가중치를 로드해줘야 합니다.

`*.safetensor` 형태의 저장된 모델 가중치로부터 가중치를 불러와 모델의 가중치에 매핑하는 함수를 구현합니다.


> - (참고) fused qkv kernel 에 대한 이해를 위해 아래의 글을 참고하시면 좋습니다.
    - [transformer](https://www.stephendiehl.com/posts/mlir_transformers/): transformer 구조에 대해 설명합니다. attnetion 에 대한 그림을 보았을 때, qkv fused kernel 을 사용하면 데이터 I/O 을 줄일 수 있을 것 입니다.
    - [grouped query attention](https://www.geeksforgeeks.org/deep-learning/grouped-query-attention-gqa/) : grouped query attention (GQA) 에 대한 설명을 담고 있습니다. 왜 q, k, v projection 에 대한 weight 을 다르게 합치는지를 이해할 수 있습니다.

In [30]:
def load_and_map_weights_for_sequence_model(model, checkpoint_path, config):
    """
    .safetensors 체크포인트와 모델 간의 가중치를 정교하게 매핑하여 로드합니다.
    """
    model_state_dict = model.state_dict()

    state = {}
    with safe_open(checkpoint_path, framework="pt") as fp:
        for k in fp.keys():
            assert k not in state
            state[k] = fp.get_tensor(k)

    print(f'model tensors: ', model_state_dict.keys())
    print('checkpoint tensors: ', state.keys())

    new_state_dict = OrderedDict()

    # 문제 5: 위에 출력한 모델 가중치의 키를 기반으로 새로운 가중치 딕셔너리를 만듭니다.
    # [START CODE]

    # 최상위 레벨 레이어 매핑
    new_state_dict['tok_embedding.weight'] = state['model.embed_tokens.weight']
    new_state_dict['final_norm.weight'] = state['model.norm.weight']

    # score weight 을 로드합니다.
    new_state_dict['score.weight'] = state['score.weight']

    # Transformer 블록 내부 레이어 매핑
    for i in range(model.config.num_layers):
        new_state_dict[f'transformer_blocks.{i}.pre_atten_norm.weight'] = state[f'model.layers.{i}.input_layernorm.weight']
        new_state_dict[f'transformer_blocks.{i}.post_atten_norm.weight'] = state[f'model.layers.{i}.post_attention_layernorm.weight']
        new_state_dict[f'transformer_blocks.{i}.atten_func.output_projection.weight'] = state[f'model.layers.{i}.self_attn.o_proj.weight']
        new_state_dict[f'transformer_blocks.{i}.ff.w1.weight'] = state[f'model.layers.{i}.mlp.gate_proj.weight']
        new_state_dict[f'transformer_blocks.{i}.ff.w2.weight'] = state[f'model.layers.{i}.mlp.down_proj.weight']
        new_state_dict[f'transformer_blocks.{i}.ff.w3.weight'] = state[f'model.layers.{i}.mlp.up_proj.weight']

        q_weight = state[f'model.layers.{i}.self_attn.q_proj.weight']
        k_weight = state[f'model.layers.{i}.self_attn.k_proj.weight']
        v_weight = state[f'model.layers.{i}.self_attn.v_proj.weight']

        attn_config = config.block_config(i).attn_config
        if attn_config.qkv_fused_interleaved:
            q_per_kv = attn_config.num_heads // attn_config.num_query_groups
            qs = torch.split(q_weight, attn_config.head_dim * q_per_kv)
            ks = torch.split(k_weight, attn_config.head_dim)
            vs = torch.split(v_weight, attn_config.head_dim)
            qkv_combined_weight = torch.cat([t for group in zip(qs, ks, vs) for t in group])
        else:
            qkv_combined_weight = torch.cat([q_weight, k_weight, v_weight], dim=0)
        new_state_dict[f'transformer_blocks.{i}.atten_func.qkv_projection.weight'] = qkv_combined_weight
    # [END CODE]

    # 모델 가중치를 로드합니다.
    model.load_state_dict(new_state_dict, strict=True)
    print("safetensors 가중치를 성공적으로 매핑하여 모델에 로드했습니다!")
    return model

다음으로 구현한 함수를 통해 모델의 가중치를 불러오고 추론을 실행해보도록 하겠습니다.

In [31]:
# 모델을 생성하고 가중치를 불러옵니다.
config = get_model_config()
model = build_model(config)
loaded_model = load_and_map_weights_for_sequence_model(model,
                                                       f'{smollm_checkpoint_path}/model.safetensors',
                                                       config)
loaded_model.eval() # 추론 모드로 설정

# --- 1. 토크나이저 로드 ---
# 사용자의 환경에 맞게 수정 필요: 사용자의 환경에 맞게 토크나이저 이름과 모델 경로를 수정하세요.
tokenizer_path = smollm_checkpoint_path
tokenizer = AutoTokenizer.from_pretrained(smollm_checkpoint_path)
print("토크나이저 로드 완료.")

# --- 2. 추론 실행 ---
prompts = [
    "회원가입은 어떻게 하나요?",
    "주문한 상품의 배송 상태를 추적하고 싶습니다.",
    "결제 시 사용 가능한 할인 쿠폰이 있나요?",
    "로그인 시도 시 오류 코드가 발생하는데 해결 방법은 무엇인가요?",
    "주문 후 결제 수단을 변경할 수 있나요?",
    "이 제품의 상세 스펙과 사용 후기를 알고 싶습니다.",
]

# --- 3. 레이블 맵 설정 ---
# 모델의 출력 인덱스(0, 1)를 실제 레이블 이름으로 매핑합니다.
# 이 부분은 사용자가 학습시킨 분류 작업에 맞게 수정해야 합니다.
tokenizer.id2label = {0: "simple", 1: "complex"}

for prompt in prompts:
    inputs = tokenizer(prompt,
                        return_tensors='pt',
                        padding='max_length',
                        truncation=True,
                        max_length=128)

    # 문제 6: 위에 생성한 모델로 추론을 수행합니다.
    # [START CODE]
    result = loaded_model(**inputs)
    # [END CODE]

    logits = result['logits'][0]
    predicted_class_id = torch.argmax(logits).item()

    print(f"입력: {prompt}")
    print(f'예측: {tokenizer.id2label[predicted_class_id]} ({logits})')

model tensors:  odict_keys(['tok_embedding.weight', 'transformer_blocks.0.pre_atten_norm.weight', 'transformer_blocks.0.atten_func.qkv_projection.weight', 'transformer_blocks.0.atten_func.output_projection.weight', 'transformer_blocks.0.post_atten_norm.weight', 'transformer_blocks.0.ff.w1.weight', 'transformer_blocks.0.ff.w2.weight', 'transformer_blocks.0.ff.w3.weight', 'transformer_blocks.1.pre_atten_norm.weight', 'transformer_blocks.1.atten_func.qkv_projection.weight', 'transformer_blocks.1.atten_func.output_projection.weight', 'transformer_blocks.1.post_atten_norm.weight', 'transformer_blocks.1.ff.w1.weight', 'transformer_blocks.1.ff.w2.weight', 'transformer_blocks.1.ff.w3.weight', 'transformer_blocks.2.pre_atten_norm.weight', 'transformer_blocks.2.atten_func.qkv_projection.weight', 'transformer_blocks.2.atten_func.output_projection.weight', 'transformer_blocks.2.post_atten_norm.weight', 'transformer_blocks.2.ff.w1.weight', 'transformer_blocks.2.ff.w2.weight', 'transformer_blocks.2.

다음으로 **PyTorch 모델을 TFLite로 변환하는 함수를 정의**합니다. 변환 과정은 다음과 같은 단계로 이루어집니다.

1. PyTorch 모델 빌드: 사전 학습된 '몸통' 가중치를 포함한 분류 모델을 준비합니다. (미세 조정한 분류 헤드 가중치는 변환 직전에 별도로 로드합니다.)

2. 더미 입력 생성: TFLite가 모델의 입출력 형태(shape)를 파악하고 그래프를 추적(trace)할 수 있도록, 실제 입력과 동일한 형태의 더미(dummy) 텐서를 생성합니다.

3. 모델 변환: `ai_edge_torch.signature`로 모델의 서명(signature)을 정의하고, `.convert()` 메서드를 호출하여 TFLite 모델로 변환합니다.

4. 파일로 내보내기: 변환된 모델을 `.tflite` 파일로 저장합니다.

In [32]:
import os

def convert_classifier_to_tflite(
    # 모델 빌드에 필요한 파라미터
    pytorch_model: nn.Module, # 가중치가 모두 로드된 모델을 직접 받음
    max_len: int = 128,
    quantize_model: bool = True,
    output_path: str = "smollm_classifier.tflite"
):
    """
    SequenceClassificationModel을 TFLite로 변환합니다.

    Args:
        pytorch_model (nn.Module): 가중치가 모두 로드된 PyTorch 모델.
        max_len (int): TFLite 모델이 받을 최대 시퀀스 길이.
        quantize_model (bool): 모델 양자화 여부.
        output_path (str): 저장될 TFLite 파일의 경로.
    """
    print(f"'{output_path}' 파일로 TFLite 변환을 시작합니다...")

    # 1. 변환 중 모델 그래프를 추적(trace)하기 위한 더미 입력(dummy input)을 생성합니다.
    #    TFLite 모델은 고정된 입력 크기를 가지므로, 배포 환경에서 사용할 최대 길이(max_len)로 설정합니다.
    tokens = torch.zeros((1, max_len), dtype=torch.long)
    attention_mask = torch.ones((1, max_len), dtype=torch.long)

    # 2. (선택 사항) 모델의 양자화(quantization) 설정을 정의합니다.
    #    양자화는 모델의 가중치를 float32에서 int8 등으로 낮춰 크기를 줄이고 추론 속도를 높이는 기술입니다.
    quant_config = quant_recipes.full_int8_dynamic_recipe() if quantize_model else None

    # 3. ai_edge_torch를 사용하여 모델을 변환합니다.
    edge_model = (
        ai_edge_torch.signature(
            "serving_default",
            pytorch_model,
            # 문제 7: 그래프를 추적하기 위해 더미 입력을 입력합니다.
            # 모델과 함께 더미 입력을 전달하여 그래프를 추적합니다.
            # [START CODE]
            (tokens, attention_mask)
            # [END CODE]
        )
        # .convert() 메서드가 실제 변환을 수행합니다.
        .convert(quant_config=quant_config)
    )

    # 4. TFLite 파일로 내보냅니다.
    edge_model.export(output_path)
    print(f"성공적으로 변환하여 '{output_path}'에 저장했습니다.")

이제 모든 준비가 끝났습니다. 다음 단계를 거쳐 실제 모델 변환을 실행합니다.

1. 사전 학습된 가중치가 있는 체크포인트 경로를 설정합니다.
2. `build_model_v2` 함수를 사용해 트랜스포머 몸통 부분의 가중치가 로드된 모델을 생성합니다.
3. 분류 작업에 맞게 미세 조정한 score 레이어의 가중치를 불러와 모델에 덮어씌웁니다.
4. `convert_classifier_to_tflite` 함수를 호출하여 최종적으로 TFLite 모델을 생성하고 저장합니다.


In [33]:
# --- TFLite 변환 실행 ---

output_f32_tflite_path = f"{root}/smollm2_135m_classifier_f32.tflite"

# 모델을 TFLite로 변환합니다.
convert_classifier_to_tflite(
    pytorch_model=loaded_model,
    max_len=128,                # TFLite 모델의 최대 입력 길이
    quantize_model=False,       # 양자화 비활성화 (float32 모델 생성)
    output_path=output_f32_tflite_path
)

'/content/drive/MyDrive/smollm2_135m_classifier_f32.tflite' 파일로 TFLite 변환을 시작합니다...


/usr/local/lib/python3.12/dist-packages/torch/export/_unlift.py:75: UserWarning: Attempted to insert a get_attr Node with no underlying reference in the owning GraphModule! Call GraphModule.add_submodule to add the necessary submodule, GraphModule.add_parameter to add the necessary Parameter, or nn.Module.register_buffer to add the necessary buffer
  getattr_node = gm.graph.get_attr(lifted_node)
/usr/local/lib/python3.12/dist-packages/torch/fx/graph.py:1801: UserWarning: Node mask_cache target mask_cache mask_cache of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torch/export/_unlift.py:75: UserWarning: Attempted to insert a get_attr Node with no underlying reference in the owning GraphModule! Call GraphModule.add_submodule to add the necessary submodule, GraphModule.add_parameter to add the necessary Parameter, or nn.Module.register_buffer to add the necessary buffer
 

Cause: Unable to locate the source code of <function canonicalize_signatures.<locals>.signature_wrapper at 0x7c0fe7c2b060>. Note that functions defined in certain environments, like the interactive Python shell, do not expose their source code. If that is the case, you should define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.experimental.do_not_convert. Original error: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


ConverterError: Could not translate MLIR to FlatBuffer.<unknown>:0: error: loc(fused["XlaCallModule:", "XlaCallModule@__inference_inner_1076"]): 'tf.XlaCallModule' op is neither a custom op nor a flex op
<unknown>:0: note: loc(fused["XlaCallModule:", "XlaCallModule@__inference_inner_1076"]): see current operation: %214 = "tf.XlaCallModule"(%213, %110, %135, %146, %157, %168, %179, %190, %201, %212, %10, %21, %32, %43, %54, %65, %76, %87, %98, %109, %121, %126, %127, %128, %129, %130, %131, %132, %133, %134, %136, %137, %138, %139, %140, %141, %142, %143, %144, %145, %147, %148, %149, %150, %151, %152, %153, %154, %155, %156, %158, %159, %160, %161, %162, %163, %164, %165, %166, %167, %169, %170, %171, %172, %173, %174, %175, %176, %177, %178, %180, %181, %182, %183, %184, %185, %186, %187, %188, %189, %191, %192, %193, %194, %195, %196, %197, %198, %199, %200, %202, %203, %204, %205, %206, %207, %208, %209, %210, %211, %0, %1, %2, %3, %4, %5, %6, %7, %8, %9, %11, %12, %13, %14, %15, %1

In [ ]:
output_i8_tflite_path = f"{root}/smollm2_135m_classifier_i8.tflite"

# 모델을 TFLite로 변환합니다.
convert_classifier_to_tflite(
    pytorch_model=loaded_model,
    max_len=128,                # TFLite 모델의 최대 입력 길이
    quantize_model=True,       # 양자화 비활성화 (float32 모델 생성)
    output_path=output_i8_tflite_path
)

In [ ]:
# 저장된 tflite 파일들의 크기를 확인해봅니다.

size_in_bytes = os.path.getsize(output_f32_tflite_path)
size_in_mb = size_in_bytes / (1024 * 1024)
print(f"파일 크기 (MB): {size_in_mb:.2f} MB")

size_in_bytes = os.path.getsize(output_i8_tflite_path)
size_in_mb = size_in_bytes / (1024 * 1024)
print(f"파일 크기 (MB): {size_in_mb:.2f} MB")

# **실습2: 변환된 모델을 tflite interpreter 을 통해 실행**

이 파트에서는 이전에 PyTorch 모델에서 변환한 .tflite 파일을 불러와, 텍스트에 대한 분류를 수행하는 전체 과정을 학습합니다.

TensorFlow Lite Interpreter를 사용하여 모델을 로드하고, 입력 데이터를 전처리하여 추론을 실행한 뒤, 최종 결과를 해석하는 방법을 단계별로 알아봅니다.

</br>

**학습 목표**:
  - TensorFlow Lite Interpreter를 사용하여 TFLite 모델을 로드하는 방법을 이해합니다.
  - 모델 추론을 위한 입력 데이터 전처리 과정을 학습합니다.
  - TFLite 모델의 signature_runner를 사용하여 추론을 실행하는 방법을 파악합니다.
  - 모델의 출력(logits)을 해석하여 최종 예측 레이블과 신뢰도 점수를 얻는 방법을 이해합니다.



모델을 실행하고 데이터를 처리하는 데 필요한 **라이브러리들을 임포트**합니다.

In [ ]:
import numpy as np
from tensorflow import lite
import time

TFLite 모델을 더 편리하게 사용하기 위해, 모델 로딩, 데이터 전처리, 추론, 후처리 과정을 하나로 묶는 래퍼(wrapper) 클래스 **LiteRTClassifier를 정의**합니다.


In [ ]:
import numpy as np
from tensorflow import lite

class LiteRTClassifier:
    """TFLite 시퀀스 분류 모델을 위한 래퍼(wrapper) 클래스."""

    def __init__(self, tfl_path: str, tokenizer, max_len: int = 128):
        """
        클래스를 초기화하고 TFLite 모델과 토크나이저를 로드합니다.

        Args:
            tfl_path (str): .tflite 모델 파일의 경로.
            tokenizer: Hugging Face의 사전 학습된 토크나이저.
            max_len (int): 모델이 처리할 수 있는 최대 시퀀스 길이.
        """
        # --- 1. 토크나이저 및 모델 설정 초기화 ---
        self.tok = tokenizer
        self.max_len = max_len

        # --- 2. TFLite 모델 로드 ---
        self.interp = lite.Interpreter(model_path=tfl_path)
        self.runner = self.interp.get_signature_runner('serving_default')

        # --- 3. 모델의 입력 이름 동적 확인 ---
        # 하드코딩 대신 모델 시그니처에서 직접 입력 키 이름을 가져옵니다.
        # 이렇게 하면 모델 변환 시 이름이 'args_0', 'args_1' 등으로 바뀌어도 코드를 수정할 필요가 없습니다.
        input_details = self.runner.get_input_details()
        self.input_keys = list(input_details.keys())

        # 모델이 두 개의 입력을 받는지 확인합니다.
        if len(self.input_keys) != 2:
            raise ValueError(
                f"TFLite 모델이 2개의 입력을 받을 것으로 예상했지만, {len(self.input_keys)}개를 받습니다. "
                f"입력 이름: {self.input_keys}"
            )

    def predict(self, text: str):
        """주어진 텍스트를 분류합니다."""

        # --- 단계 1: 입력 데이터 전처리 (Preprocessing) ---
        # 토크나이저는 'input_ids'와 'attention_mask'를 모두 반환합니다.
        inputs = self.tok(
            text,
            max_length=self.max_len,
            padding="max_length",
            truncation=True,
            return_tensors="np"
        )

        # TFLite 모델의 입력 타입(int64)으로 변환합니다.
        tokens = inputs["input_ids"].astype("int64")
        attention_mask = inputs["attention_mask"].astype("int64")

        # --- 단계 2: 모델 입력 텐서 매핑 ---
        # 동적으로 찾은 입력 이름에 맞게 tokens와 attention_mask를 매핑합니다.
        # 첫 번째 입력은 tokens, 두 번째는 attention_mask로 가정합니다.
        # 문제 8: 모델의 입력에 맞게 텐서를 매핑시킵니다.
        # [START CODE]
        feed_dict = {
            self.input_keys[0]: tokens,
            self.input_keys[1]: attention_mask
        }
        # [END CODE]

        # --- 단계 3: 추론 실행 (Inference) ---
        outputs = self.runner(**feed_dict)
        logits = outputs["logits"]

        # --- 단계 4: 결과 후처리 (Post-processing) ---
        predicted_class_id = int(np.argmax(logits, axis=-1)[0])
        score = float(logits[0, predicted_class_id])
        label = self.tok.id2label[predicted_class_id]

        return {
            "label": label,
            "score": score,
            'predicted_class_id': predicted_class_id,
            'logits': logits
        }

이제 LiteRTClassifier 클래스를 사용하여 실제 **텍스트에 대한 분류를 수행**해 보겠습니다.

In [ ]:
# ---  분류기 클래스 인스턴스화 ---
# 사용자의 환경에 맞게 수정 필요: 변환된 tflite 파일의 경로를 입력하세요
output_fp32_tflite_path = f"{root}/module-14/smollm2_135m_classifier_f32.tflite"
output_i8_tflite_path = f"{root}//module-14/smollm2_135m_classifier_i8.tflite"

classifier_fp32 = LiteRTClassifier(output_fp32_tflite_path, tokenizer, max_len=128)
classifier_i8 = LiteRTClassifier(output_i8_tflite_path, tokenizer, max_len=128)
print("분류기 생성 완료.\n")

In [ ]:
# --- 추론 실행 ---
prompts = [
    "회원가입은 어떻게 하나요?",
    "주문한 상품의 배송 상태를 추적하고 싶습니다.",
    "결제 시 사용 가능한 할인 쿠폰이 있나요?",
    "로그인 시도 시 오류 코드가 발생하는데 해결 방법은 무엇인가요?",
    "주문 후 결제 수단을 변경할 수 있나요?",
    "이 제품의 상세 스펙과 사용 후기를 알고 싶습니다.",
]

print("\n------float 32 model 의 예측을 수행합니다.------")
for prompt in prompts:
    start_time = time.time()
    result = classifier_fp32.predict(prompt)
    end_time = time.time()

    inference_time = end_time - start_time

    print(f"입력: '{prompt}'")
    print(f"-> 예측 결과: '{result['logits']}' '{result['label']}' (신뢰도: {result['score']:.4f})")
    print(f"   추론 시간: {inference_time:.4f} 초\n")

print("\n------int 8 model 의 예측을 수행합니다.------")
for prompt in prompts:
    start_time = time.time()
    result = classifier_i8.predict(prompt)
    end_time = time.time()

    inference_time = end_time - start_time

    print(f"입력: '{prompt}'")
    print(f"-> 예측 결과: '{result['logits']}' '{result['label']}' (신뢰도: {result['score']:.4f})")
    print(f"   추론 시간: {inference_time:.4f} 초\n")

Colab 환경의 일반 CPU에서 테스트했을 때 float32와 int8 모델의 속도 차이가 기대만큼 드라마틱하게 차이나지 않는데요. 이는 자연스러운 현상입니다.

이는 int8 연산의 진짜 힘이 NPU, TPU, 모바일 GPU와 같은 특화된 하드웨어 가속기에서 발휘되기 때문입니다.

CPU 는 소수의 매우 복잡하고 강력한 코어를 가지고 있습니다. 각 코어는 어려운 문제를 순서대로 처리하는 데 특화되어 있으며, 특히 소수점 연산(Float32)을 위한 전용 회로(FPU, Floating-Point Unit)가 고도로 발달해 있습니다.

반대로  NPU/TPU는 수천, 수만 개의 매우 단순하고 작은 연산 유닛(MAC, Multiply-Accumulate)으로 구성되어 있습니다. 이 유닛들은 복잡한 작업은 못 하지만, 단순한 정수(INT8) 곱셈과 덧셈을 대규모로 동시에(in parallel) 처리하는 능력은 뛰어납니다.

Colab의 CPU는 이미 Float32 연산에 매우 뛰어나기 때문에, 그보다 단순한 INT8 연산을 처리해도 속도 향상 폭이 크지 않습니다. 하지만 스마트폰에 탑재된 NPU는 설계 단계부터 수많은 코어가 INT8 연산을 동시에 처리하도록 만들어졌기 때문에, INT8로 양자화된 모델을 만나면 하드웨어의 잠재력이 증가하여 극적인 속도 향상과 함께 전력 소모량도 크게 줄일 수 있을 것 입니다.

결론적으로 Colab 환경에서 INT8 모델의 주된 이점은 획기적으로 줄어든 모델 파일 크기와 감소된 메모리 사용량입니다. 약간의 속도 향상은 덤이라고 생각할 수 있습니다. 드라마틱한 속도 향상은 실제 타겟 디바이스(스마트폰 등)에 배포했을 때 비로소 체감할 수 있습니다.

